In [83]:
import jieba
import numpy as np
import logging
import pandas as pd
from collections import Counter
jieba.setLogLevel(logging.INFO)


In [84]:
# 参数设置
path = "E:\\毕业设计"
k1 = 1.5
b = 0.75

In [85]:
# 获取停用词
stopwords = pd.read_csv(path + "\\GraduationDesign\\语料库\\stopwords.dat", delimiter="\t", header=None, quoting=3, encoding='utf-8')
stopwords = stopwords[0].tolist()

In [86]:
# 对三句话进行分割，存入列表
# doc_list = [doc for doc in text.split('\n') if doc != '']
# doc_list

In [87]:
# 读取客服数据
df = pd.read_csv(path + "\\GraduationDesign\\语料库\\客服语料\\整理后\\1.csv")
docs = []
for doc in df.iloc[:,0]:
    temp_list = [word for word in jieba.cut(doc) if word not in stopwords and word !=' ']
    docs.append(temp_list)
docs

[['注册', '新', '账户'],
 ['更改', '订单'],
 ['客服'],
 ['取消', '订单'],
 ['订单', '到货', '这是', '怎么回事'],
 ['登录', '情况', '浏览', '商品'],
 ['礼品', '卡', '支付'],
 ['获取', '退款'],
 ['账户', '锁定', '我该'],
 ['商品', '退换'],
 ['更改', '账户', '密码'],
 ['订单', '发货', '取消', '订单', '退款'],
 ['忘记', '用户名'],
 ['想', '公司', '产品', '推荐'],
 ['退货', '退款', '尚未', '显示', '账户', '余额', '中'],
 ['应用程序', '优惠券'],
 ['想要', '提供', '体验', '反馈', '意见', '做'],
 ['访问', '订单', '历史记录'],
 ['账户', '锁定'],
 ['购买', '商品', '质量', '我该'],
 ['更改', '密码'],
 ['退货'],
 ['更改', '订单'],
 ['网站', '多种', '货币', '结算'],
 ['订单', '发货'],
 ['客户服务', '团队'],
 ['退货', '时', '全额', '退款'],
 ['订单', '延迟', '我该'],
 ['贵', '公司', '网站', '找到', '产品', '保修', '信息'],
 ['取消', '订单'],
 ['查询', '订单', '状态'],
 ['客服'],
 ['想要', '更改', '送货', '地址'],
 ['刚刚', '收到', '订单', '商品', '损坏'],
 ['退款', '退还', '账户'],
 ['错过', '促销', '活动', '提供', '优惠', '码'],
 ['告诉', '产品', 'XX', '信息'],
 ['网站', '提交', '订单'],
 ['想要', '取消', '订单', '付出', '额外', '费用'],
 ['产品质量', '感到', '满意', '退货'],
 ['更新', '账户', '信息'],
 ['找到', '订单'],
 ['订单', '送达'],
 ['取消', '订单'],
 ['退货'],
 ['注册', '

In [88]:
# 对每句话进行分词
# docs = []
# for sentence in doc_list:
#     docs.append([word for word in jieba.cut(sentence) if word not in stopwords and word !=' '])
# docs

In [89]:
vocab = set([word for doc in docs for word in doc]) # 文档中所包含的所有词语
vocab

{'APP',
 'VIP',
 'X',
 'XX',
 '一件',
 '一步',
 '三个',
 '上个月',
 '上架',
 '下单',
 '下架',
 '不佳',
 '不到',
 '不好',
 '不符',
 '不足以',
 '丢失',
 '个人信息',
 '个人资料',
 '中',
 '买',
 '交易',
 '交货',
 '交货期',
 '产品',
 '产品价格',
 '产品描述',
 '产品线',
 '产品规格',
 '产品质量',
 '享受',
 '人员',
 '介绍',
 '付',
 '付出',
 '付款',
 '代码',
 '代表',
 '价格',
 '价格比',
 '优势',
 '优惠',
 '优惠券',
 '优惠政策',
 '优惠活动',
 '会到',
 '会员',
 '体验',
 '余额',
 '使用手册',
 '供选择',
 '促销',
 '保修',
 '保修期',
 '保修期限',
 '保证',
 '信息',
 '信用卡',
 '修改',
 '做',
 '做些',
 '做出',
 '儿童',
 '充值',
 '免费',
 '兑换',
 '全额',
 '公司',
 '具体内容',
 '内容',
 '几天',
 '分享',
 '分期付款',
 '列表',
 '刚刚',
 '删除',
 '到货',
 '到达',
 '刷',
 '功能',
 '包含',
 '包括',
 '包裹',
 '升级',
 '卡',
 '历史背景',
 '历史记录',
 '原因',
 '参加',
 '参数',
 '参数信息',
 '反馈',
 '发现',
 '发现错误',
 '发生',
 '发票',
 '发货',
 '发送',
 '发送给',
 '取得联系',
 '取消',
 '变动',
 '变化',
 '可用',
 '合作伙伴',
 '合适',
 '名',
 '告诉',
 '咨询',
 '售后',
 '售后服务',
 '商品',
 '商品价格',
 '商品信息',
 '商品质量',
 '喜欢',
 '回复',
 '团队',
 '困难',
 '国际',
 '国际标准',
 '在线',
 '地址',
 '填写',
 '增加',
 '声明',
 '处于',
 '处理结果',
 '多久',
 '多种',
 '多长',
 '多长时间',
 '失败',
 '好处',
 '婴幼儿',


In [90]:
def getWordIdf(word):
    if word not in vocab:
        word_idf = 0
    else:
        # 统计包含该词语的文档数
        qn = {}
        for doc in docs:
            if word in doc:
                if word in qn:
                    qn[word] += 1
                else:
                    qn[word] = 1
            else:
                continue
        #word_idf = np.log((len(docs) - qn[word] + 0.5) / (qn[word] + 0.5))
        word_idf = np.log((len(docs) + 1) / (qn[word] + 1))   #得到词语word的idf值
    return word_idf

In [91]:
avgdl = sum([len(doc) + 0.0 for doc in docs]) / len(docs) # 所有文档的平均长度

In [92]:
# 计算词语word在各句的得分
def getWordScoreList(word):
    score_list = []
    for doc in docs:   # 每句话
        word_count = Counter(doc)   # 统计一句话中各词语的词频
        if word in word_count.keys():   # 如果测试词语在这句话中
            f = (word_count[word]+0.0) / len(docs[0])
        else:
            f = 0.0
        r_score = (f * (k1 + 1)) / (f + k1 * (1 - b + b * len(doc) / avgdl))   # 计算词语word的r_score
        score_list.append(getWordIdf(word) * r_score)    # 计算词语word在该句中的BM25分数
    return score_list

In [93]:
def getSequenceScore(sequence):
    scores = []
    for word in sequence:
        scores.append(getWordScoreList(word))
    return np.sum(scores, axis=0)

In [94]:
# 问句处理
question = "如何更改我的订单？"
q_list = [word for word in jieba.cut(question) if word not in stopwords and word !=' ']
q_list

['更改', '订单']

In [95]:
sim = getSequenceScore(q_list)

In [96]:
# 找到sim中最大的前5个值及其索引
max_sim = []
max_index = []
for i in range(5):
    max_sim.append(max(sim))
    max_index.append(np.argmax(sim))
    sim[np.argmax(sim)] = 0
    
print(max_sim)
print(max_index)

[2.581627171362858, 2.581627171362858, 2.581627171362858, 2.581627171362858, 2.581627171362858]
[1, 22, 102, 181, 222]


In [97]:
all_answer = df.iloc[:,1]
print("匹配到问题：", df.iloc[max_index[0],0])
print("回答：", all_answer[max_index[0]])

匹配到问题：  我如何更改我的订单？ 
回答：  如果您需要更改订单，请在订单状态变为“已付款”之前联系客服人员进行更改。 
